In [199]:
import requests


subreddit = "anxiety"
after = int(datetime(2022, 11, 7).timestamp())
before = int(datetime(2022, 11, 8).timestamp())


size = 1000

url = f"https://api.pushshift.io/reddit/submission/search?subreddit={subreddit}&after={after}&before={before}&size={size}"
response = requests.get(url)
data = response.json()



In [200]:
len(data['data'])

260

In [171]:
with open("text.json", 'w', encoding='UTF-8') as f:
    json.dump(data['data'],f)

In [190]:
import requests
from datetime import datetime, timedelta
import time,json
import os.path

class SubredditScrapper:
    def __init__(self,subname = "all",batchsize = 1000,timeblock=[1,0],order = 'desc',orderby = 'created_utc',output_path=""):
        self.config = {
            'subname':subname,
            'size':min(1000,batchsize),
            'timeblock':timeblock,
            'sort': order,
            'sort_type': orderby,
        }
        self.output_path = output_path
    
    def SearchQuery(self,subreddit,start,end,size):
        return f"https://api.pushshift.io/reddit/submission/search?subreddit={subreddit}&after={start}&before={end}&size={size}"
    
    def TimeQueryFormat(self,time):
        return int(time.timestamp())
    
    def ExtractFields(self,data):
        author,title,text,subreddit,flair,postid,url,time = 8*[""]
        author = data['author_fullname'] if data['author_fullname'] else ""
        title = data['title'] if data['title'] else ""
        text = data['selftext'] if data['selftext'] else ""
        subreddit = data['subreddit'] if data['subreddit'] else ""
        flair = data['link_flair_text'] if data['link_flair_text'] else ""
        postid = data['id'] if data['id'] else ""
        url = data['url'] if data['url'] else ""
        time = data['created_utc'] if data['created_utc'] else ""
        return [author,title,text,subreddit,flair,postid,url,time]
    
    def GetData(self,query):
        response = requests.get(query)
        
        if response.status_code != 200:
            print(f"Query at '{query}' failed with status code: {response.status_code}")
            return "Error"
        data = response.json()
        posts = data['data']
        result = [self.ExtractFields(x) for x in posts]
#         print(result)
        return result
    
    def getOutputFilename(self,start,end):
        return self.output_path+self.config['subname']+"_"+ start.strftime("%m-%d-%Y") +"to"+ end.strftime("%m-%d-%Y") + ".json"
        
    def PostsBetween(self,start,end):
        
        if os.path.exists(self.getOutputFilename(start,end)):
            prompt = input("File already exists, Do you want to continue and overwrite? (y/n)")
            if prompt.lower() == "n":
                return
#         else:
#             f = open(self.getOutputFilename(start,end), "x")
        
        

        all_posts = []
        current_start = start
        prev_time = None
        
        while current_start < end:
            current_end = current_start + timedelta(hours=self.config['timeblock'][0],minutes=self.config['timeblock'][1])
            current_end = current_end if current_end < end else end
            url = self.SearchQuery(subreddit=self.config['subname'], start=self.TimeQueryFormat(current_start), end=self.TimeQueryFormat(current_end), size=self.config['size'])
#             print(url)
            data = self.GetData(url)
            if data == 'Error':
                print(f"Returning partial data.")
                return all_posts

            
            if len(data) == 0 and current_start < end:
                print(f"No posts found between {current_start} and {current_end}")
                current_start = current_end
            else:
                data.sort(key=lambda x: x['Created Time'],reverse=False)
                if prev_time == datetime.fromtimestamp(data[-1]['Created Time']):
                    # No new posts found after this start time, exit condition reached
                    return all_posts
                else:
                    all_posts.extend(data)
                    prev_time = current_start
                    current_start = datetime.fromtimestamp(data[-1]['Created Time']+1)
#             print(data)
#             print(start,current_start,end)
            # Sleep to avoid overloading the server
            time.sleep(2)
        with open(self.getOutputFilename(start,end), 'w', encoding='UTF-8') as f:
            json.dump(all_posts, f)
        return all_posts


In [153]:
sub = SubredditScrapper('mentalhealth')
after = datetime(2022, 11, 3)
before = datetime(2022, 11, 5)
sub.PostsBetween(after,before)

No posts found between 2022-11-03 00:00:00 and 2022-11-03 01:00:00
No posts found between 2022-11-03 01:00:00 and 2022-11-03 02:00:00
No posts found between 2022-11-03 02:00:00 and 2022-11-03 03:00:00
No posts found between 2022-11-03 03:00:00 and 2022-11-03 04:00:00
No posts found between 2022-11-03 04:00:00 and 2022-11-03 05:00:00
No posts found between 2022-11-03 05:00:00 and 2022-11-03 06:00:00
No posts found between 2022-11-03 06:00:00 and 2022-11-03 07:00:00
No posts found between 2022-11-03 07:00:00 and 2022-11-03 08:00:00
No posts found between 2022-11-03 08:00:00 and 2022-11-03 09:00:00
No posts found between 2022-11-03 09:00:00 and 2022-11-03 10:00:00
No posts found between 2022-11-03 10:00:00 and 2022-11-03 11:00:00
No posts found between 2022-11-03 11:00:00 and 2022-11-03 12:00:00
No posts found between 2022-11-04 23:57:42 and 2022-11-05 00:00:00


[{'Author': 't2_ty2gd2p4',
  'Title': 'Porn at the age of 6',
  'Text': 'Before you comment "stop asking reddit for help and seek medical professional help", understand that I already did, and the solutions they gave me did do something, but not in the most mindfulness/healthy manner. \n\nAt the age of 6/7 I was exposed to porn and I feel like that ruined the way I am. My self confidence and behavorial/mental health has been all over the place because of it. As of a couple years ago, I realized that I am actually very attractive to the average person, but I think why I couldn\'t believe that myself or get 100% into my comfort zone was because i\'m awkward, with ADHD, and possibly bipolar, which I think mostly stems from the porn exposure. But here I am at 25 with the knowledge that I have now. I feel as if it is too late to be that person I want to be, unless I get help. So here I am. Looking for alternatives.\n\nI wont rely mainly on **porn** for my social demise, but I think it plays

In [100]:
data['data'][0].values()

dict_values(['mentalhealth', 'Throwaway account.\n\nI am 18 Years old. Just now noticing just how much hate I have. I don’t know why I am like this. I tend to do things and say things to put people down and I enjoy it. I dont enjoy it in a good way if that makes sense. When I’m doing it I dont notice but after the fact its just sad. \n\nAgain Idc how this post is layed out just wanna vent and maybe get some answers or help\n\nI work at a grocery store and if someone gets in my way I just start insulting them in my head. Why do I do that? Its like I feed of negativity for no reason. \n\nI find joy when other people are struggling with something. I go out of my way to make peoples days worse. I put myself in front of other people to make my day better and theres worse. I seek to piss people off.\n\nIts not even with other people. I scare myself. I was looking at myself in the mirror into my own eyes, I got scared. Its like I seen a monster. I am disgusted with myself. I’m scared of the t

In [148]:
import pandas as pd

In [149]:
pd.read_json('mentalhealth_11-03-2022to11-04-2022.json')

,Title,Text,Subreddit,Flair,Post ID,URL,Created Time
0,Porn at the age of 6,"Before you comment ""stop asking reddit for hel...",mentalhealth,Need Support,ylcb5o,https://www.reddit.com/r/mentalhealth/comments...,1667503955
1,I just feel like I need to say some things to ...,[removed],mentalhealth,Venting,ylcb9e,https://www.reddit.com/r/mentalhealth/comments...,1667503963
2,Abandonment Issues?,Is it possible to develop abandonment issues i...,mentalhealth,Question,ylcbts,https://www.reddit.com/r/mentalhealth/comments...,1667504004
3,Struggling with a voice !,I am dealing with a voice in my head that says...,mentalhealth,Need Support,ylcerr,https://www.reddit.com/r/mentalhealth/comments...,1667504210
4,My mom is in a really bad mental state.,"So my Mom is in a depression, she used to work...",mentalhealth,Venting,ylch6k,https://www.reddit.com/r/mentalhealth/comments...,1667504370
...,...,...,...,...,...,...,...
157,Where do I belong?,"Continuing my journey of face to face classes,...",mentalhealth,None,ylqww5,https://www.reddit.com/r/mentalhealth/comments...,1667543522
158,Am I The Problem?,i just finished an appointment with the third ...,mentalhealth,Opinion / Thoughts,ylr3ma,https://www.reddit.com/r/mentalhealth/comments...,1667544239
159,What do you find embarrassing about your menta...,Is there anything about your diagnostic label ...,mentalhealth,Question,ylr3u9,https://www.reddit.com/r/mentalhealth/comments...,1667544262
160,I had a near-death-experience 1.5 years ago. I...,I arrived to US to complete a PhD program in C...,mentalhealth,None,ylr72e,https://www.reddit.com/r/mentalhealth/comments...,1667544602


In [207]:
pd.read_json('../data/mentalhealth11-03-2022to02-16-2023.json')

,Author,Title,Text,Subreddit,Flair,Post ID,Url,Created Time
0,t2_ty2gd2p4,Porn at the age of 6,"Before you comment ""stop asking reddit for hel...",mentalhealth,Need Support,ylcb5o,https://www.reddit.com/r/mentalhealth/comments...,1667503955
1,t2_c5khkmri,I just feel like I need to say some things to ...,[removed],mentalhealth,Venting,ylcb9e,https://www.reddit.com/r/mentalhealth/comments...,1667503963
2,t2_38y569z9,Abandonment Issues?,Is it possible to develop abandonment issues i...,mentalhealth,Question,ylcbts,https://www.reddit.com/r/mentalhealth/comments...,1667504004
3,t2_sl92wb1o,Struggling with a voice !,I am dealing with a voice in my head that says...,mentalhealth,Need Support,ylcerr,https://www.reddit.com/r/mentalhealth/comments...,1667504210
4,t2_8ha9qccf,My mom is in a really bad mental state.,"So my Mom is in a depression, she used to work...",mentalhealth,Venting,ylch6k,https://www.reddit.com/r/mentalhealth/comments...,1667504370
...,...,...,...,...,...,...,...,...
28459,t2_crhuo26l,Why does this bother me so bad!!??,i absolutely hate and cannot stand when people...,mentalhealth,Venting,113kyqn,https://www.reddit.com/r/mentalhealth/comments...,1676532789
28460,t2_19qx2771,Being Alone.,"First post here. Tried searching the internet,...",mentalhealth,Opinion / Thoughts,113kzd7,https://www.reddit.com/r/mentalhealth/comments...,1676532867
28461,t2_1u5myicz,Selfish? Help me become selfless,"In the last 5 years, I’ve heard twice that I w...",mentalhealth,None,113kzmb,https://www.reddit.com/r/mentalhealth/comments...,1676532896
28462,t2_guc5zfri,Is this a sign of PTSD or mild psychosis?,So I just wanted to see what you guys think. D...,mentalhealth,None,113l1so,https://www.reddit.com/r/mentalhealth/comments...,1676533140


In [208]:
pd.read_json('../data/anxiety11-03-2022to02-16-2023.json')

,Author,Title,Text,Subreddit,Flair,Post ID,Url,Created Time
0,t2_6374r84,Spiraling about friend who hasnt responded to ...,"Hello, so for whatever reason my anxiety alway...",Anxiety,Needs A Hug/Support,ylcl5y,https://www.reddit.com/r/Anxiety/comments/ylcl...,1667504595
1,t2_ty09vokd,This loneliness I have makes me feel really al...,\n. I am 19 years old I made a pledge to mysel...,Anxiety,Venting,ylcm49,https://www.reddit.com/r/Anxiety/comments/ylcm...,1667504652
2,t2_st8j7myu,feeling overwhelmed and disappointed with myself.,I feel disappointed with myself. I feel disapp...,Anxiety,Venting,ylcu62,https://www.reddit.com/r/Anxiety/comments/ylcu...,1667505140
3,t2_ke9rzqzf,"what over counter pill, medications help with ...","what over counter pill, medications help with...",Anxiety,Medication,ylcx7j,https://www.reddit.com/r/Anxiety/comments/ylcx...,1667505321
4,t2_tdl2d7xu,Weight loss,"Hi, \nCan anxiety/fear cause weight loss on it...",Anxiety,Advice Needed,ylcyqo,https://www.reddit.com/r/Anxiety/comments/ylcy...,1667505413
...,...,...,...,...,...,...,...,...
22579,t2_z9cvhqd,Overwhelming flight or fight response to couns...,I've been in counselling since November but ha...,Anxiety,Advice Needed,113ksce,https://www.reddit.com/r/Anxiety/comments/113k...,1676532090
22580,t2_aozcyuch,Does Clonazepam affect your memory?,I took .25mg of Clonazepam yesterday night and...,Anxiety,Advice Needed,113kxor,https://www.reddit.com/r/Anxiety/comments/113k...,1676532678
22581,t2_508xfmz1,"so idk if this is an anxiety question, but hon...","So like idk, right now I'm really light on fri...",Anxiety,Advice Needed,113l3ma,https://www.reddit.com/r/Anxiety/comments/113l...,1676533352
22582,t2_4ndfpdln,For about 4/5 days now my sleep has been poor....,"I was sleeping okay until about 5 days ago, th...",Anxiety,Advice Needed,113l6sd,https://www.reddit.com/r/Anxiety/comments/113l...,1676533722


In [209]:
pd.read_json('../data/depression11-03-2022to02-16-2023.json')

,Author,Title,Text,Subreddit,Flair,Post ID,Url,Created Time
0,t2_6ammjhsk,depression sucks,Depression sucks but I get so much joy from da...,depression,NaN,ylcaxy,https://www.reddit.com/r/depression/comments/y...,1667503939
1,t2_2dpho4ca,"As much as I want to say that I don't care, it...","I have little to no friends, decided to cut of...",depression,NaN,ylce4r,https://www.reddit.com/r/depression/comments/y...,1667504166
2,t2_7fap7djo,Is there any coming back/recovery after being ...,Title sums it up more or less. But here’s a de...,depression,NaN,ylcf7j,https://www.reddit.com/r/depression/comments/y...,1667504240
3,t2_988wuslk,looking for hope,I've been major depressive for most of my life...,depression,NaN,ylcjvd,https://www.reddit.com/r/depression/comments/y...,1667504522
4,t2_ty09vokd,I’m going to commit suicide tonight going to b...,\nI am 19 years old I made a pledge to myself ...,depression,NaN,ylckaz,https://www.reddit.com/r/depression/comments/y...,1667504545
...,...,...,...,...,...,...,...,...
34708,t2_sv5laujf,i need to talk to someone,[removed],depression,NaN,113ksfu,https://www.reddit.com/r/depression/comments/1...,1676532100
34709,t2_sv5laujf,parents,\n\nher parents said we cant get in contact a...,depression,NaN,113l0eo,https://www.reddit.com/r/depression/comments/1...,1676532979
34710,t2_7i10jke2,"""You should talk to your friends/family about ...",[removed],depression,NaN,113l0le,https://www.reddit.com/r/depression/comments/1...,1676533001
34711,t2_r454c40u,What’s the point of it all?,"Why do people live, what’s her point? We all e...",depression,NaN,113l2or,https://www.reddit.com/r/depression/comments/1...,1676533241
